In [1]:
%load_ext autoreload
%autoreload 2

In [26]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('../raw_data/raw_en.csv')

df.info()

/tmp/ipykernel_8863/1251664015.py:6: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../raw_data/raw_en.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457074 entries, 0 to 457073
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   name            457074 non-null  object 
 1   date            351179 non-null  object 
 2   recommendation  350678 non-null  object 
 3   review          457074 non-null  object 
 4   playing_hours   350678 non-null  float64
 5   source          457074 non-null  object 
 6   stars           106396 non-null  float64
 7   language        457074 non-null  object 
dtypes: float64(2), object(6)
memory usage: 27.9+ MB


In [3]:
df['language'].value_counts()

en    457074
Name: language, dtype: int64

In [28]:
df1 = df.head(10000).copy()
# creating small sample to train with

In [29]:
from customerchurn.pre_pipeline import create_pre_pipe
from customerchurn.pre_pipeline import series_prepro
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer



pipe = create_pre_pipe()

pipe.fit(df1)

df1['review_clean'] = pipe.transform(df1)[:,0]

In [30]:
df2 = df.head(10000).copy()
# creating small sample to train with

In [23]:
# !pip install -q -U tensorflow-text
# !pip install -q tf-models-official

You should consider upgrading via the '/home/cathal/.pyenv/versions/3.8.6/envs/customerchurn/bin/python -m pip install --upgrade pip' command.


In [3]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')


2022-02-05 13:04:19.254092: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-05 13:04:19.254147: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/cathal/.pyenv/versions/3.8.6/envs/customerchurn/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.5.0 and strictly below 2.8.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the Ten

In [33]:
df2.recommendation.value_counts()

Recommended        6578
Not Recommended    3422
Name: recommendation, dtype: int64

In [34]:
df2.head()

,name,date,recommendation,review,playing_hours,source,stars,language
0,Expansion - Hearts of Iron IV: Man the Guns,2019-02-10,Recommended,&gt Played as German Reich&gt Declare war on B...,578.0,steam,NaN,en
1,Expansion - Hearts of Iron IV: Man the Guns,2019-02-07,Recommended,Very good game although a bit overpriced in my...,892.0,steam,NaN,en
2,Dead by Daylight,2018-06-14,Recommended,Out of all the reviews I wrote This one is pro...,676.0,steam,NaN,en
3,Dead by Daylight,2017-06-20,Recommended,Disclaimer I survivor main. I play games for f...,612.0,steam,NaN,en
4,Dead by Daylight,2016-12-12,Recommended,ENGLISH After playing for more than two years ...,2694.0,steam,NaN,en


In [52]:
X= df2['review']
y = df2['recommendation'].apply(lambda x: 0 if x == 'Not Recommended' else 1)

In [37]:
y.value_counts()

1    6578
0    3422
Name: recommendation, dtype: int64

In [ ]:
# AUTOTUNE = tf.data.AUTOTUNE
# batch_size = 32
# seed = 42

# raw_train_df = tf.keras.utils.text_dataset_from_directory(
#     'customer/train',
#     batch_size=batch_size,
#     validation_split=0.2,
#     subset='training',
#     seed=seed)

# class_names = raw_train_ds.class_names
# train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

# val_ds = tf.keras.utils.text_dataset_from_directory(
#     'customer/train',
#     batch_size=batch_size,
#     validation_split=0.2,
#     subset='validation',
#     seed=seed)

# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# test_ds = tf.keras.utils.text_dataset_from_directory(
#     'customer/test',
#     batch_size=batch_size)

# test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [10]:
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [13]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
tfhub_handle_preprocess= 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [11]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

2022-02-05 14:30:01.860601: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-05 14:30:01.860661: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-05 14:30:01.860819: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-1CG9GS9): /proc/driver/nvidia/version does not exist
2022-02-05 14:30:01.861315: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_type_ids', 'input_mask', 'input_word_ids']
Shape      : (1, 128)
Word Ids   : [ 101 2023 2003 2107 2019 6429 3185  999  102    0    0    0]
Input Mask : [1 1 1 1 1 1 1 1 1 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [14]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [15]:
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Pooled Outputs Shape:(1, 512)
Pooled Outputs Values:[ 0.76262903  0.9928099  -0.1861187   0.3667383   0.15233696  0.65504414
  0.9681154  -0.9486272   0.00216167 -0.9877732   0.06842678 -0.9763059 ]
Sequence Outputs Shape:(1, 128, 512)
Sequence Outputs Values:[[-0.28946355  0.34321314  0.3323151  ...  0.2130079   0.7102079
  -0.05771136]
 [-0.28742105  0.31981066 -0.2301857  ...  0.58455014 -0.21329743
   0.7269204 ]
 [-0.6615694   0.6887674  -0.8743296  ...  0.10877217 -0.26173162
   0.4785526 ]
 ...
 [-0.22561109 -0.28925592 -0.07064441 ...  0.4756602   0.83277166
   0.40025324]
 [-0.29824194 -0.2747312  -0.05450527 ...  0.48849782  1.0955361
   0.18163359]
 [-0.4437828   0.00930762  0.07223727 ...  0.17290081  1.1833241
   0.07897999]]


In [16]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')

    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')

    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']

    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)

    return tf.keras.Model(text_input, net)
# Activiation should be sigmoid

In [17]:
classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(text_test))
print(tf.sigmoid(bert_raw_result))

tf.Tensor([[0.41161916]], shape=(1, 1), dtype=float32)


In [43]:
# loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
# metrics = tf.metrics.BinaryAccuracy()

In [51]:
# epochs = 5
# steps_per_epoch = tf.data.experimental.cardinality(X).numpy()
# num_train_steps = steps_per_epoch * epochs
# num_warmup_steps = int(0.1*num_train_steps)

# init_lr = 3e-5
# optimizer = optimization.create_optimizer(init_lr=init_lr,
#                                           num_train_steps=num_train_steps,
#                                           num_warmup_steps=num_warmup_steps,
#                                           optimizer_type='adamw')

AttributeError: 'numpy.ndarray' object has no attribute '_variant_tensor'

In [54]:
classifier_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

classifier_model.fit(X,y)

313/313 [==============================] - 3063s 10s/step - loss: 5.2264 - accuracy: 0.6564


In [19]:
!pip install pydot

You should consider upgrading via the '/home/cathal/.pyenv/versions/3.8.6/envs/customerchurn/bin/python -m pip install --upgrade pip' command.


In [21]:
# !sudo apt install graphviz

[sudo] password for cathal: 


In [24]:
tf.keras.utils.plot_model(classifier_model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
